<a href="https://colab.research.google.com/github/zxcasd945/Video_compression/blob/main/motion_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import cv2
import numpy as np

#Video sequences url
football_url = "https://media.xiph.org/video/derf/y4m/football_422_cif.y4m"
tennis_url = "https://media.xiph.org/video/derf/y4m/tennis_sif.y4m"
garden_url = "https://media.xiph.org/video/derf/y4m/garden_sif.y4m"

def full_search(reference_frame, current_frame, block_size=16, search_range=7):
    height, width = reference_frame.shape
    motion_vectors = np.zeros((height // block_size, width // block_size, 2), dtype=int)

    total_checkpoints = 0  # Variable to count the total number of check points

    for i in range(0, height, block_size):
        for j in range(0, width, block_size):
            block_reference = reference_frame[i:i + block_size, j:j + block_size]
            best_match = None
            min_mse = float('inf')
            current_checkpoints = 0  # Variable to count the number of check points for each motion vector

            for m in range(max(0, i - search_range), min(height - block_size, i + search_range + 1)):
                for n in range(max(0, j - search_range), min(width - block_size, j + search_range + 1)):
                    current_checkpoints += 1
                    block_current = current_frame[m:m + block_size, n:n + block_size]
                    mse = np.sum((block_reference - block_current) ** 2)

                    if mse < min_mse:
                        min_mse = mse
                        best_match = (m - i, n - j)

            motion_vectors[i // block_size, j // block_size] = best_match
            total_checkpoints += current_checkpoints

    average_checkpoints_per_mv = total_checkpoints / (motion_vectors.shape[0] * motion_vectors.shape[1])
    print(f"Average Checkpoints per Motion Vector: {average_checkpoints_per_mv}")

    return motion_vectors
def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    return 10 * np.log10((max_pixel ** 2) / mse)

# Load the video sequence
cap = cv2.VideoCapture(garden_url)
ret, frame1 = cap.read()
frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Initialize variables for PSNR calculation
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_psnr = 0.0

# Perform motion estimation for each frame
for frame_count in range(1, total_frames):
    ret, frame2 = cap.read()
    if not ret:
        break

    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    motion_vectors = full_search(frame1_gray, frame2_gray)

    reconstructed_frame = np.zeros_like(frame2_gray)
    for i in range(0, frame2_gray.shape[0], 16):
        for j in range(0, frame2_gray.shape[1], 16):
            mv = motion_vectors[i // 16, j // 16]
            reconstructed_frame[i:i + 16, j:j + 16] = frame1_gray[i + mv[0]:i + mv[0] + 16, j + mv[1]:j + mv[1] + 16]

    current_psnr = psnr(frame2_gray, reconstructed_frame)
    total_psnr += current_psnr

    # Update the reference frame for the next iteration
    frame1_gray = frame2_gray

average_psnr = total_psnr / (total_frames - 1)
print(f"Average PSNR: {average_psnr} dB")

# Release the video capture object
cap.release()

Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkpoints per Motion Vector: 200.45454545454547
Average Checkp

In [21]:
import cv2
import numpy as np

#Video sequences url
football_url = "https://media.xiph.org/video/derf/y4m/football_422_cif.y4m"
tennis_url = "https://media.xiph.org/video/derf/y4m/tennis_sif.y4m"
garden_url = "https://media.xiph.org/video/derf/y4m/garden_sif.y4m"

def three_step_search(reference_frame, current_frame, block_size=16, search_range=7):
    height, width = reference_frame.shape
    motion_vectors = np.zeros((height // block_size, width // block_size, 2), dtype=int)

    total_checkpoints = 0  # Variable to count the total number of check points

    for i in range(0, height, block_size):
        for j in range(0, width, block_size):
            block_reference = reference_frame[i:i + block_size, j:j + block_size]
            best_match = None
            min_mse = float('inf')
            step_size = search_range // 2
            current_checkpoints = 0  # Variable to count the number of check points for each motion vector

            while step_size >= 1:
                for m in range(max(0, i - step_size), min(height - block_size, i + step_size + 1), step_size):
                    for n in range(max(0, j - step_size), min(width - block_size, j + step_size + 1), step_size):
                        current_checkpoints += 1
                        block_current = current_frame[m:m + block_size, n:n + block_size]
                        mse = np.sum((block_reference - block_current) ** 2)

                        if mse < min_mse:
                            min_mse = mse
                            best_match = (m - i, n - j)

                step_size = step_size // 2

            motion_vectors[i // block_size, j // block_size] = best_match
            total_checkpoints += current_checkpoints

    average_checkpoints_per_mv = total_checkpoints / (motion_vectors.shape[0] * motion_vectors.shape[1])
    print(f"Average Checkpoints per Motion Vector: {average_checkpoints_per_mv}")

    return motion_vectors


# Use the three-step search motion estimation function
motion_vectors = three_step_search(frame1_gray, frame2_gray)
def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    return 10 * np.log10((max_pixel ** 2) / mse)

# Load the video sequence
cap = cv2.VideoCapture(garden_url)
ret, frame1 = cap.read()
frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Initialize variables for PSNR calculation
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_psnr = 0.0

# Perform motion estimation for each frame
for frame_count in range(1, total_frames):
    ret, frame2 = cap.read()
    if not ret:
        break

    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    motion_vectors = three_step_search(frame1_gray, frame2_gray)
    # You can use the motion vectors for further processing or analysis

    reconstructed_frame = np.zeros_like(frame2_gray)
    for i in range(0, frame2_gray.shape[0], 16):
        for j in range(0, frame2_gray.shape[1], 16):
            mv = motion_vectors[i // 16, j // 16]
            reconstructed_frame[i:i + 16, j:j + 16] = frame1_gray[i + mv[0]:i + mv[0] + 16, j + mv[1]:j + mv[1] + 16]

    current_psnr = psnr(frame2_gray, reconstructed_frame)
    total_psnr += current_psnr

    # Update the reference frame for the next iteration
    frame1_gray = frame2_gray

average_psnr = total_psnr / (total_frames - 1)
print(f"Average PSNR: {average_psnr} dB")

# Release the video capture object
cap.release()

Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkpoints per Motion Vector: 16.036363636363635
Average Checkp

In [22]:
import cv2
import numpy as np

#Video sequences url
football_url = "https://media.xiph.org/video/derf/y4m/football_422_cif.y4m"
tennis_url = "https://media.xiph.org/video/derf/y4m/tennis_sif.y4m"
garden_url = "https://media.xiph.org/video/derf/y4m/garden_sif.y4m"

def diamond_search(reference_frame, current_frame, block_size=16, search_range=7):
    height, width = reference_frame.shape
    motion_vectors = np.zeros((height // block_size, width // block_size, 2), dtype=int)

    total_checkpoints = 0  # Variable to count the total number of check points

    for i in range(0, height, block_size):
        for j in range(0, width, block_size):
            block_reference = reference_frame[i:i + block_size, j:j + block_size]
            best_match = (0, 0)  # Initialize with (0, 0) as the default best match
            min_mse = np.inf
            step_size = search_range // 2
            current_search_point = [i, j]
            current_checkpoints = 0  # Variable to count the number of check points for each motion vector

            while step_size >= 1:
                for offset in [(0, -step_size), (0, step_size), (-step_size, 0), (step_size, 0)]:
                    m, n = current_search_point[0] + offset[0], current_search_point[1] + offset[1]
                    current_checkpoints += 1  # Increment the check point counter
                    if 0 <= m < height - block_size and 0 <= n < width - block_size:
                        block_current = current_frame[m:m + block_size, n:n + block_size]
                        mse = np.sum((block_reference - block_current) ** 2)

                        if mse < min_mse:
                            min_mse = mse
                            best_match = (m - i, n - j)

                current_search_point[0] += best_match[0]
                current_search_point[1] += best_match[1]
                step_size = step_size // 2

            motion_vectors[i // block_size, j // block_size] = best_match
            total_checkpoints += current_checkpoints

    average_checkpoints_per_mv = total_checkpoints / (motion_vectors.shape[0] * motion_vectors.shape[1])
    print(f"Average Checkpoints per Motion Vector: {average_checkpoints_per_mv}")

    return motion_vectors

# Use the diamond search motion estimation function
motion_vectors = diamond_search(frame1_gray, frame2_gray)
def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    return 10 * np.log10((max_pixel ** 2) / mse)

# Load the video sequence
cap = cv2.VideoCapture(garden_url)
ret, frame1 = cap.read()
frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Initialize variables for PSNR calculation
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_psnr = 0.0

# Perform motion estimation for each frame
for frame_count in range(1, total_frames):
    ret, frame2 = cap.read()
    if not ret:
        break

    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    motion_vectors = diamond_search(frame1_gray, frame2_gray)
    # You can use the motion vectors for further processing or analysis

    reconstructed_frame = np.zeros_like(frame2_gray)
    for i in range(0, frame2_gray.shape[0], 16):
        for j in range(0, frame2_gray.shape[1], 16):
            mv = motion_vectors[i // 16, j // 16]
            reconstructed_frame[i:i + 16, j:j + 16] = frame1_gray[i + mv[0]:i + mv[0] + 16, j + mv[1]:j + mv[1] + 16]

    current_psnr = psnr(frame2_gray, reconstructed_frame)
    total_psnr += current_psnr

    # Update the reference frame for the next iteration
    frame1_gray = frame2_gray

average_psnr = total_psnr / (total_frames - 1)
print(f"Average PSNR: {average_psnr} dB")

# Release the video capture object
cap.release()

Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Checkpoints per Motion Vector: 8.0
Average Che